In [1]:
import sounddevice as sd
from scipy.io.wavfile import write
from tkinter import Tk, Button, Label
import threading

class Recorder:
    def __init__(self, fs=44100):
        self.fs = fs
        self.is_recording = False
        self.frames = []

    def start_recording(self):
        self.is_recording = True
        print("Recording started...")
        self.frames = []

        def callback(indata, frames, time, status):
            if self.is_recording:
                self.frames.append(indata.copy())

        self.stream = sd.InputStream(callback=callback, channels=1, samplerate=self.fs)
        self.stream.start()

    def stop_recording(self, filename):
        if not self.is_recording:
            return
        
        self.is_recording = False
        print("Recording stopped.")
        self.stream.stop()
        self.stream.close()
        
        recording = np.concatenate(self.frames)
        write(filename, self.fs, recording)
        print(f"File saved as: {filename}")

def create_app():
    recorder = Recorder()

    def start():
        threading.Thread(target=recorder.start_recording).start()

    def stop():
        filename = 'test.wav'
        threading.Thread(target=recorder.stop_recording, args=(filename,)).start()

    root = Tk()

    Label(root, text="Audio Recorder").pack()
    Button(root, text="Start", command=start).pack()
    Button(root, text="Stop", command=stop).pack()

    root.mainloop()

create_app()
